In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_df = pd.read_csv("../../input/feedback-prize-english-language-learning/train.csv")
train_df.head(2)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


In [3]:
model_path_list = [
    ('../../14_Baseline4/exp/result/14_v1_01/oof_df.csv',1), # deberta-v3-base
    ('../../14_Baseline4/exp/result/14_v1_08/oof_df.csv',1), # deberta-base
    ('../../14_Baseline4/exp/result/14_v1_09/oof_df.csv',1), # roberta-base
    ('../../14_Baseline4/exp/result/14_v1_10/oof_df.csv',1), # deberta-v3-large
    ('../../14_Baseline4/exp/result/14_v1_11/oof_df.csv',1), # xlm-roberta-base
    ('../../14_Baseline4/exp/result/14_v1_12/oof_df.csv',1), # deberta-large
    ('../../14_Baseline4/exp/result/14_v1_13/oof_df.csv',1), # roberta-large
    ('../../14_Baseline4/exp/result/14_v1_14/oof_df.csv',1), # xlm-roberta-large
    ('../../14_Baseline4/exp/result/14_v1_15/oof_df.csv',1), # deberta-v2-xlarge
    ('../../14_Baseline4/exp/result/14_v1_16/oof_df.csv',1), # deberta-xlarge
    
    ('../../14_Baseline4/exp/result/14_v2_01/oof_df.csv',1), # deberta-v3-base, seed200
    ('../../14_Baseline4/exp/result/14_v2_02/oof_df.csv',1), # deberta-v3-large, seed200
    
    #('../../17_Pseudo3/exp/result/17_v1_01/oof_df.csv',1), # deberta-v3-base
    #('../../17_Pseudo3/exp/result/17_v1_02/oof_df.csv',1), # deberta-v3-base
]

oof_df_list = [
    pd.read_csv(model_path) for model_path,_ in model_path_list
]
weights_list = [
    w for _,w in model_path_list
]
weights_list = [w/sum(weights_list) for w in weights_list]

for oof_df in oof_df_list:
    oof_df = train_df[['text_id']].merge(oof_df, how='left', on='text_id')
    
num_models = len(model_path_list)

TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

preds = 0
for oof_df, w in zip(oof_df_list, weights_list):
    preds += oof_df[TARGET_COLS].values * w

oofs = []
for oof_df in oof_df_list:
    oofs.append(oof_df[TARGET_COLS].values)

In [4]:
oof_df = oof_df_list[0].copy()
for i,col in enumerate(TARGET_COLS):
    oof_df[col] = preds[:,i]

In [5]:
import numpy as np

def calc_metric(pred, gt):
    '''
    pred : (num_data, num_labels)
    gt : (num_data, num_labels)
    '''
    score = np.sqrt(np.mean((pred - gt)**2, axis=0))
    score = score.mean()
    return score

In [6]:
score = calc_metric(pred=oof_df[TARGET_COLS].values, gt=train_df[TARGET_COLS].values)
print('CV={:.4f}'.format(score))

CV=0.4439


# Optimize class-wise

In [7]:
ensemble_predictions=np.stack(oofs)
ensemble_predictions.shape

(12, 3911, 6)

In [8]:
from skopt import gp_minimize

def run_optimize(i):
    target_col = TARGET_COLS[i]
    print("*"*50)
    print(target_col)
    print("*"*50)
    def ensemble_score(weights,return_pred=False):
        weights=np.array(weights)
        weights=weights.reshape(-1,1,1)/weights.sum()
        p=weights.reshape(-1,1,1)*ensemble_predictions[:,:,i,None]
        p=p.sum(0)
        score=calc_metric(p, train_df[[target_col]])
        if return_pred:
            return score,p
        else:
            return score   
    results = gp_minimize(ensemble_score, np.array([[0.1,1] for i in range(len(ensemble_predictions))]),
                          verbose=True,random_state=2022)
    best_weights=np.array(results['x'])/sum(results['x'])
    print(best_weights)
    # smaller is better for this metric
    score, ensemble_pred=ensemble_score(best_weights,True)
    print("score={:.4f}", score)
    return score, best_weights

In [9]:
scores = {}
best_weights = {}
for i,target_col in enumerate(TARGET_COLS):
    score, weights = run_optimize(i)
    scores[target_col] = score
    best_weights[target_col] = weights

**************************************************
cohesion
**************************************************
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0085
Function value obtained: 0.4715
Current minimum: 0.4715
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0023
Function value obtained: 0.4713
Current minimum: 0.4713
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0023
Function value obtained: 0.4721
Current minimum: 0.4713
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0024
Function value obtained: 0.4725
Current minimum: 0.4713
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.4643
Function value obtained: 0.4717
Current minimum: 0.4707
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 0.5137
Function value obtained: 0.4711
Current minimum: 0.4707
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.4164
Function value obtained: 0.4726
Current minimum: 0.4707
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 0.4593
Function value obtained: 0.4711
Current minimum: 0.4707
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.4985
Function value obtained: 0.4725
Current minimum: 0.4707
Iteration No: 46 started. Sea

Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 0.7351
Function value obtained: 0.4720
Current minimum: 0.4706
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 0.7284
Function value obtained: 0.4706
Current minimum: 0.4706
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 0.7486
Function value obtained: 0.4720
Current minimum: 0.4706
Iteration No: 84 started. Searching for the next optimal point.
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 0.7234
Function value obtained: 0.4729
Current minimum: 0.4706
Iteration No: 85 started. Searching for the next optimal point.
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 0.7560
Function value obtained: 0.4734
Current minimum: 0.4706
Iteration No: 86 started. Sea

Iteration No: 20 ended. Search finished for the next optimal point.
Time taken: 0.4760
Function value obtained: 0.4367
Current minimum: 0.4367
Iteration No: 21 started. Searching for the next optimal point.
Iteration No: 21 ended. Search finished for the next optimal point.
Time taken: 0.3995
Function value obtained: 0.4368
Current minimum: 0.4367
Iteration No: 22 started. Searching for the next optimal point.
Iteration No: 22 ended. Search finished for the next optimal point.
Time taken: 0.4269
Function value obtained: 0.4376
Current minimum: 0.4367
Iteration No: 23 started. Searching for the next optimal point.
Iteration No: 23 ended. Search finished for the next optimal point.
Time taken: 0.5507
Function value obtained: 0.4379
Current minimum: 0.4367
Iteration No: 24 started. Searching for the next optimal point.
Iteration No: 24 ended. Search finished for the next optimal point.
Time taken: 0.3657
Function value obtained: 0.4371
Current minimum: 0.4367
Iteration No: 25 started. Sea

Iteration No: 60 ended. Search finished for the next optimal point.
Time taken: 0.5995
Function value obtained: 0.4368
Current minimum: 0.4366
Iteration No: 61 started. Searching for the next optimal point.
Iteration No: 61 ended. Search finished for the next optimal point.
Time taken: 0.6208
Function value obtained: 0.4378
Current minimum: 0.4366
Iteration No: 62 started. Searching for the next optimal point.
Iteration No: 62 ended. Search finished for the next optimal point.
Time taken: 0.5870
Function value obtained: 0.4368
Current minimum: 0.4366
Iteration No: 63 started. Searching for the next optimal point.
Iteration No: 63 ended. Search finished for the next optimal point.
Time taken: 0.6498
Function value obtained: 0.4373
Current minimum: 0.4366
Iteration No: 64 started. Searching for the next optimal point.
Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 0.7744
Function value obtained: 0.4371
Current minimum: 0.4366
Iteration No: 65 started. Sea

...

Iteration No: 37 ended. Search finished for the next optimal point.
Time taken: 0.4410
Function value obtained: 0.4626
Current minimum: 0.4624
Iteration No: 38 started. Searching for the next optimal point.
Iteration No: 38 ended. Search finished for the next optimal point.
Time taken: 0.5934
Function value obtained: 0.4624
Current minimum: 0.4624
Iteration No: 39 started. Searching for the next optimal point.
Iteration No: 39 ended. Search finished for the next optimal point.
Time taken: 0.4658
Function value obtained: 0.4650
Current minimum: 0.4624
Iteration No: 40 started. Searching for the next optimal point.
Iteration No: 40 ended. Search finished for the next optimal point.
Time taken: 0.5301
Function value obtained: 0.4626
Current minimum: 0.4624
Iteration No: 41 started. Searching for the next optimal point.
Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.4953
Function value obtained: 0.4637
Current minimum: 0.4624
Iteration No: 42 started. Sea

Iteration No: 77 ended. Search finished for the next optimal point.
Time taken: 0.6603
Function value obtained: 0.4651
Current minimum: 0.4622
Iteration No: 78 started. Searching for the next optimal point.
Iteration No: 78 ended. Search finished for the next optimal point.
Time taken: 0.6657
Function value obtained: 0.4624
Current minimum: 0.4622
Iteration No: 79 started. Searching for the next optimal point.
Iteration No: 79 ended. Search finished for the next optimal point.
Time taken: 0.6507
Function value obtained: 0.4622
Current minimum: 0.4622
Iteration No: 80 started. Searching for the next optimal point.
Iteration No: 80 ended. Search finished for the next optimal point.
Time taken: 0.7042
Function value obtained: 0.4641
Current minimum: 0.4622
Iteration No: 81 started. Searching for the next optimal point.
Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 0.6638
Function value obtained: 0.4650
Current minimum: 0.4622
Iteration No: 82 started. Sea

Iteration No: 16 ended. Search finished for the next optimal point.
Time taken: 0.2897
Function value obtained: 0.4376
Current minimum: 0.4372
Iteration No: 17 started. Searching for the next optimal point.
Iteration No: 17 ended. Search finished for the next optimal point.
Time taken: 0.2878
Function value obtained: 0.4373
Current minimum: 0.4372
Iteration No: 18 started. Searching for the next optimal point.
Iteration No: 18 ended. Search finished for the next optimal point.
Time taken: 0.3546
Function value obtained: 0.4374
Current minimum: 0.4372
Iteration No: 19 started. Searching for the next optimal point.
Iteration No: 19 ended. Search finished for the next optimal point.
Time taken: 0.3055
Function value obtained: 0.4375
Current minimum: 0.4372
Iteration No: 20 started. Searching for the next optimal point.
Iteration No: 20 ended. Search finished for the next optimal point.
Time taken: 0.3631
Function value obtained: 0.4373
Current minimum: 0.4372
Iteration No: 21 started. Sea

Iteration No: 56 ended. Search finished for the next optimal point.
Time taken: 0.6705
Function value obtained: 0.4373
Current minimum: 0.4370
Iteration No: 57 started. Searching for the next optimal point.
Iteration No: 57 ended. Search finished for the next optimal point.
Time taken: 0.5308
Function value obtained: 0.4370
Current minimum: 0.4370
Iteration No: 58 started. Searching for the next optimal point.
Iteration No: 58 ended. Search finished for the next optimal point.
Time taken: 0.5990
Function value obtained: 0.4378
Current minimum: 0.4370
Iteration No: 59 started. Searching for the next optimal point.
Iteration No: 59 ended. Search finished for the next optimal point.
Time taken: 0.5261
Function value obtained: 0.4370
Current minimum: 0.4370
Iteration No: 60 started. Searching for the next optimal point.
Iteration No: 60 ended. Search finished for the next optimal point.
Time taken: 0.6570
Function value obtained: 0.4372
Current minimum: 0.4370
Iteration No: 61 started. Sea

Iteration No: 96 ended. Search finished for the next optimal point.
Time taken: 1.0154
Function value obtained: 0.4369
Current minimum: 0.4368
Iteration No: 97 started. Searching for the next optimal point.
Iteration No: 97 ended. Search finished for the next optimal point.
Time taken: 1.0536
Function value obtained: 0.4368
Current minimum: 0.4368
Iteration No: 98 started. Searching for the next optimal point.
Iteration No: 98 ended. Search finished for the next optimal point.
Time taken: 0.9758
Function value obtained: 0.4376
Current minimum: 0.4368
Iteration No: 99 started. Searching for the next optimal point.
Iteration No: 99 ended. Search finished for the next optimal point.
Time taken: 0.9537
Function value obtained: 0.4368
Current minimum: 0.4368
Iteration No: 100 started. Searching for the next optimal point.
Iteration No: 100 ended. Search finished for the next optimal point.
Time taken: 0.9682
Function value obtained: 0.4378
Current minimum: 0.4368
[0.0822366  0.01411533 0.01

In [10]:
best_weights

{'cohesion': array([0.06015702, 0.05269822, 0.01816616, 0.04596064, 0.01816616,
        0.13183059, 0.01816616, 0.01816616, 0.10616082, 0.16907013,
        0.1797964 , 0.18166155]),
 'syntax': array([0.13878434, 0.01701662, 0.01701662, 0.14559043, 0.01701662,
        0.17016616, 0.04599196, 0.13425343, 0.13171234, 0.01701662,
        0.05183641, 0.11359847]),
 'vocabulary': array([0.19477603, 0.02186533, 0.02186533, 0.20121873, 0.07739327,
        0.02186533, 0.02186533, 0.08025188, 0.09651488, 0.02186533,
        0.02186533, 0.21865325]),
 'phraseology': array([0.10108329, 0.01471234, 0.01471234, 0.14712339, 0.05288088,
        0.14712339, 0.01471234, 0.01471234, 0.14712339, 0.05156951,
        0.14712339, 0.14712339]),
 'grammar': array([0.08545829, 0.02239537, 0.02239537, 0.22395375, 0.0370602 ,
        0.02239537, 0.02239537, 0.02239537, 0.10537593, 0.02239537,
        0.18982583, 0.22395375]),
 'conventions': array([0.0822366 , 0.01411533, 0.01411533, 0.0812697 , 0.07827566,
     

In [11]:
scores

{'cohesion': 0.47060277226222536,
 'syntax': 0.4365709772383802,
 'vocabulary': 0.4042194862575178,
 'phraseology': 0.44583525060133306,
 'grammar': 0.46218872440226344,
 'conventions': 0.4368299485165353}

In [12]:
CV = sum(scores.values()) / len(scores)
print("CV={:.4f}".format(CV))

CV=0.4427


# Error Analysis - Check Corr

In [13]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
train_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.695459,0.666151,0.690058,0.638689,0.666151
syntax,0.695459,1.000000,0.680562,0.725467,0.709525,0.700025
vocabulary,0.666151,0.680562,1.000000,0.735261,0.654852,0.664292
phraseology,0.690058,0.725467,0.735261,1.000000,0.719746,0.666842
grammar,0.638689,0.709525,0.654852,0.719746,1.000000,0.673301
conventions,0.666151,0.700025,0.664292,0.666842,0.673301,1.000000


In [14]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
oof_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.945868,0.914444,0.911507,0.825909,0.886977
syntax,0.945868,1.000000,0.937416,0.960180,0.916072,0.928610
vocabulary,0.914444,0.937416,1.000000,0.958528,0.874674,0.891235
phraseology,0.911507,0.960180,0.958528,1.000000,0.942623,0.880940
grammar,0.825909,0.916072,0.874674,0.942623,1.000000,0.855358
conventions,0.886977,0.928610,0.891235,0.880940,0.855358,1.000000
